# Generate Data For Analysis of Glint Visibility

This notebook generates data for analysis of glint visibility.
Generation of 100 combined user-device samples x two eyes x 20 gaze directions,
i.e., 4,000 individual samples, takes about 25-40 min on a beefy machine.

In [ ]:
%matplotlib widget

import datetime
import seet
import os
import pickle
import sys
import torch
import pandas

sys.path.append("..")
import utils

dropdown_widget, text_widget = utils.get_experiment_info()

## Data Generation

In [ ]:
def generate_data_for_visibility_analysis(scene_sampler, gaze_grid=[5, 4]):
    """generate_data_for_visibility_analysis.

    Generate data for visibility analysis.

    Args:
        gaze_grid (list, optional): size of gaze direction grid on which to
        sample the gaze directions. First element is the number of
        horizontal samples, second element is the number of vertical
        samples. Defaults to [5, 4].
    """

    # We want to collect data about LED visibility, working distance, eye
    # clipping, gaze direction, eye relief.

    #######################################################################
    # Device-only data
    header_subsystem = ["Subsystem", ]
    header_grid_angle = ["Horiz. angle", "Vert. angle"]
    # This assumes that the subsystems have the same number of LEDs.
    num_leds = scene_sampler.scene.device.subsystems[0].led_set.num
    header_LEDs = ["LED {:02d}".format(i) for i in range(1, num_leds + 1)]

    #######################################################################
    # Device + user data.
    header_camera_center_in_pupil = \
        [
            "Camera center in pupil {:s}".format(ax)
            for ax in ["x", "y", "z"]
        ]
    header_delta_eye_relief = ["Delta eye relief", ]
    header_scene_index = ["Scene index", ]

    #######################################################################
    # User-only data.
    header_gaze_direction = \
        ["Gaze {:s}".format(ax) for ax in ["x", "y", "z"]]
    header_IPD = ["IPD", ]

    #######################################################################
    # Putting it all together
    header = \
        header_subsystem + \
        header_grid_angle + \
        header_LEDs + \
        header_camera_center_in_pupil + \
        header_delta_eye_relief + \
        header_scene_index + \
        header_gaze_direction + \
        header_IPD

    num_subsystems = len(scene_sampler.scene.device.subsystems)
    data = []
    for scene_index, et_scene in enumerate(scene_sampler.generate_samples()):
        for subsystem_index in range(num_subsystems):
            ###############################################################
            # Device-only data.
            # Subsystem data.
            row_subsystem = [subsystem_index, ]

            # Grid angle data.
            if gaze_grid[0] != 1 or gaze_grid[1] != 1:
                fov_range_deg = scene_sampler.scene.device.display_fov / 2
                h_fov_range_deg = \
                    torch.linspace(
                        -fov_range_deg[0], fov_range_deg[0], gaze_grid[0]
                    )
                v_fov_range_deg = \
                    torch.linspace(
                        -fov_range_deg[1], fov_range_deg[1], gaze_grid[1]
                    )
                rotate = True
            else:
                h_fov_range_deg = torch.zeros(1)
                v_fov_range_deg = torch.zeros(1)
                rotate = False

            subsystem = et_scene.device.subsystems[subsystem_index]

            camera_index = 0  # In the future, we may have stereo.
            camera = subsystem.cameras[camera_index]

            eye = et_scene.user.eyes[subsystem_index]

            for hi in range(gaze_grid[0]):
                for vi in range(gaze_grid[1]):
                    # Rotate the eye if required.
                    if rotate:
                        angles_deg = \
                            torch.stack(
                                (h_fov_range_deg[hi], v_fov_range_deg[vi])
                            )
                        eye.rotate_from_gaze_angles_inParent(angles_deg)
                    else:
                        angles_deg = torch.zeros(2)

                    row_grid_angle = \
                        [*angles_deg.clone().detach().numpy()]

                    #######################################################
                    # Device plus user data.
                    # Scene (device + user) index.

                    # LED-visibility data.
                    glints_inCamera = \
                        et_scene.generate_glints_inOther(
                            other_node=camera,
                            subsystem_index=subsystem_index,
                            camera_index=camera_index
                        )

                    row_LEDs = \
                        [int(g is not None) for g in glints_inCamera]

                    # Camera center in pupil.
                    transform_toPupil_fromCamera = \
                        camera.get_transform_toOther_fromSelf(eye.pupil)
                    optical_center_in_pupil = \
                        transform_toPupil_fromCamera.transform(
                            torch.zeros(3)
                        )
                    row_camera_center_in_pupil = \
                        [*optical_center_in_pupil.clone().detach().numpy()]

                    # Eye-relief data.
                    eye_relief_plane = subsystem.eye_relief_plane
                    cornea_apex_inPlane = \
                        eye.get_cornea_apex_inOther(eye_relief_plane)

                    delta_eye_relief = \
                        -1 * eye_relief_plane.\
                        compute_signed_distance_to_point_inPlane(
                            cornea_apex_inPlane
                        ).clone().detach().numpy()

                    row_delta_eye_relief = [delta_eye_relief, ]

                    # Keeping track of the samples.
                    row_scene_index = [scene_index, ]

                    #######################################################
                    # User-only data.
                    # Gaze-direction data.
                    gaze_direction_inScene = \
                        eye.get_gaze_direction_inOther(et_scene)
                    row_gaze_direction = \
                        [*gaze_direction_inScene.clone().detach().numpy()]

                    # IPD data.
                    IPD_data = et_scene.user.compute_IPD()
                    row_IPD = [IPD_data.clone().detach().numpy(), ]

                    #######################################################
                    # Putting it all together.
                    row = \
                        row_subsystem + \
                        row_grid_angle + \
                        row_LEDs + \
                        row_camera_center_in_pupil + \
                        row_delta_eye_relief + \
                        row_scene_index + \
                        row_gaze_direction + \
                        row_IPD

                    data = data + [row, ]

                    if rotate:
                        eye.unrotate_from_gaze_angles_inParent(angles_deg)

    return pandas.DataFrame(data, columns=header)

In [ ]:
scene_file_name, \
    sampler_file_name = \
    utils.get_configuration_files(dropdown_widget.value)

# print("Scene generated using " + scene_file_name + " configuration file.")
et_scene = seet.scene.SceneModel(parameter_file_name=scene_file_name)

print("Sampling parameters from " + sampler_file_name + " configuration file.")
scene_sampler = seet.sampler.SceneSampler(
    et_scene, num_samples=100, parameter_file_name=sampler_file_name)

df = generate_data_for_visibility_analysis(scene_sampler)

now = datetime.datetime.now()
prefix = now.strftime("%Y-%m-%d @ %H-%M-%S.%f")
results_path = text_widget.value
os.makedirs(results_path, exist_ok=True)
path_prefix = os.path.join(results_path, prefix)

df_name = path_prefix + " data_frame.pkl"
with open(df_name, 'wb') as file_stream:
    pickle.dump(df, file_stream)

df